In [ ]:
###############################################################################
#                             CHATBOT GINA - ITALIAN GRANDMA                  #
###############################################################################
# most of the structure for the chatbot was taken from Mauro Di Pietro blogs and
# https://github.com/mdipietro09/GenerativeAI/blob/main/Chatbot/chatbot.py

# Setup
## module for speech-to-text
import speech_recognition as sr 

## module for text-to-speech
from gtts import gTTS 

## use ollama for LLMs
import ollama 

## change the model below if needed. (need to pull it in ollama)
model = "qwen2.5"

## describe the personality of the chatbot and 
## instructions on how to answer. 
## This will be sent to the LLM as system prompt

system_prompt = f'''Your name is Grandma Gina. 
You are an italian grandma, sarcastic to the point of being cynical. 
You always make fun of everyone when answering the questions. 
If someone needs a cheer up, you offer to prepare some traditional italian dish, like lasagna or carbonara. 
When you refer to yourself you always use the third person. 
Your responses always include only two sentences.'''

## for data
import os
import numpy as np


# Build the ChatBot
class ChatBot():
    def __init__(self, name):
        print("--- starting up", name, "---")
        self.name = name.lower()
        self.text = ""

    def speech_to_text(self):
        recognizer = sr.Recognizer()
        with sr.Microphone() as mic:
            recognizer.adjust_for_ambient_noise(mic, duration=1)
            print("listening...")
            audio = recognizer.listen(mic)
        try:
            self.text = recognizer.recognize_google(audio)
            print("me --> ", self.text)
        except:
            print("me -->  ERROR")

    @staticmethod
    def text_to_speech(text):
        print("ai --> ", text)
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("res.mp3")
        os.system("afplay res.mp3")  #mac->afplay | windows->start
        os.remove("res.mp3")

    ## predetermined commands
    def wake_up(self, text):
        lst = ["wake up "+self.name, self.name+"wake up ", "hey "+self.name]
        return True if any(i in ai.text.lower() for i in lst) else False

    def what(self, text):
        lst = ["what are you", "who are you"]
        return True if any(i in ai.text.lower() for i in lst) else False

    

# Run the AI
if __name__ == "__main__":
    
    ai = ChatBot(name="Grandma Gina")
    #ai.text = ""
    
    while True:
        ai.speech_to_text()

        ## wake up
        if ai.wake_up(ai.text) is True:
            res = "Hello I am Grandma Gina, how are you doing?"

        ## what
        elif ai.what(ai.text) is True:
            res = "I am an AI created by Angelo."

        
        ## respond politely
        elif any(i in ai.text for i in ["thank","thanks"]):
            res = np.random.choice(["you're welcome!","no problema!","Grandma Gina is here if you need her!"])
        
        ## general conversation
        else:   
            
            prompt = [{"role":"system", "content":system_prompt}]
            prompt = prompt.append( {"role":"user", "content":ai.text} ) 
           
            res = ollama.generate(model=model, prompt=system_prompt + ai.text)["response"]
            res = res.split("\n")[0]

        ai.text_to_speech(res)
        